In [1]:
import pandas as pd
import numpy as np
import requests
import swifter

In [2]:
# Read in 1.2 Million Spotify Songs dataset
spotify_1m = pd.read_csv('./Resources/tracks_features.csv')

In [3]:
# Strip brackets and single quotes from artist name
spotify_1m['artist'] = spotify_1m['artists'].str.strip("[']")

# Preview dataframe
spotify_1m.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,artist
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02,Rage Against The Machine
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02,Rage Against The Machine
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02,Rage Against The Machine
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02,Rage Against The Machine
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02,Rage Against The Machine


In [48]:
# Read in Billboard Top 100 dataset
df = pd.read_csv('./Resources/charts.csv')

# Convert date column to date type
df['date'] = pd.to_datetime(df['date'])
# Create new column for year
df['year'] = df['date'].dt.year
# Remove special characters from artist and song name
df['artist'] = df['artist'].str.replace(r"\(.*\)","")
# df['song'] = df['song'].str.replace(r"\(.*\)","")

df['song'] = df['song'].str.replace(".","") \
                        .str.replace("\"", "") \
                        .str.replace(",", "") \
                        .str.replace("!", "")


# Create new column showing number of weeks in #1 spot (if exists)
df['weeks-at-no1'] = df[df['rank']==1].groupby(['song', 'artist', 'rank'], as_index=False).count()['date']

# Remove single quotes/apostrophes from song names
df['song'] = df['song'].str.replace("'", "")

# Preview dataframe
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,weeks-at-no1
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021,1.0
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021,7.0
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021,1.0
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021,4.0
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021,2.0


In [50]:
# Create new dataframe of all unique Billboard charting songs
unique_billboard_tracks_df = df.groupby(['song','artist', 'year'], as_index=False).agg({'peak-rank': 'min', 
                                                                                        'weeks-on-board': 'max',
                                                                                        'weeks-at-no1': 'max'})

# Create separate dataframes for each decade
billboard_1960s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1970) & (unique_billboard_tracks_df['year']>=1960)]
billboard_1970s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1980) & (unique_billboard_tracks_df['year']>=1970)]
billboard_1980s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1990) & (unique_billboard_tracks_df['year']>=1980)]
billboard_1990s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2000) & (unique_billboard_tracks_df['year']>=1990)]
billboard_2000s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2010) & (unique_billboard_tracks_df['year']>=2000)]
billboard_2010s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2020) & (unique_billboard_tracks_df['year']>=2010)]
billboard_2020s = unique_billboard_tracks_df[unique_billboard_tracks_df['year']>=2020]

# Create list of dataframes
billboard_dfs = [billboard_1960s, billboard_1970s, billboard_1980s, billboard_1990s, billboard_2000s, billboard_2010s, billboard_2020s]

# Display dataframe
unique_billboard_tracks_df.head()
unique_billboard_tracks_df

# df[(df['artist']=='Beastie Boys') & (df['year']==1986)]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
0,#1,Nelly,2001,22,11,NaN
1,#1,Nelly,2002,22,20,NaN
2,#1 Dee Jay,Goody Goody,1978,82,5,NaN
3,#9 Dream,John Lennon,1974,58,2,NaN
4,#9 Dream,John Lennon,1975,9,12,NaN
...,...,...,...,...,...,...
36078,whoknows,Musiq,2004,65,13,NaN
36079,wwwmemory,Alan Jackson,2000,45,9,NaN
36080,wwwmemory,Alan Jackson,2001,45,15,NaN
36081,¿Dònde Està Santa Claus? (Where Is Santa Claus?),Augie Rios,1958,47,3,NaN


In [53]:
unique_billboard_tracks_df[unique_billboard_tracks_df['song'].str.contains("\?")]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
105,(If You Let Me Make Love To You Then) Why Cant...,Ronnie Dyson,1970,8,14,NaN
172,(Sartorial Eloquence) Dont Ya Wanna Play This ...,Elton John,1980,39,12,NaN
1269,All Good?,De La Soul Featuring Chaka Khan,2000,96,2,NaN
1270,All Good?,De La Soul Featuring Chaka Khan,2001,96,3,NaN
1935,Any Man Of Mine/Whose Bed Have Your Boots Been...,Shania Twain,1995,31,19,NaN
...,...,...,...,...,...,...
35432,You Know I Love You Dont You?,Howard Jones,1987,17,16,NaN
36052,homecoming queen?,Kelsea Ballerini,2019,82,3,NaN
36053,homecoming queen?,Kelsea Ballerini,2020,65,13,NaN
36081,¿Dònde Està Santa Claus? (Where Is Santa Claus?),Augie Rios,1958,47,3,NaN


In [20]:
# SPOTIFY API
# Import dependencies for Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import Client ID and Client Secret
from config import cid, secret


# Create objects for accessing Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [66]:
# Create function to retrieve track ID from Spotify given the artist and song title
def get_track_id(song_title, artist):
    # Select first artist if multiple listed with "Featuring" keyword
    if 'Featuring' in artist:
        artist = artist.split(' Featuring ')[0]
    # Select first artist if multiple listed with "with" keyword
    elif ' with ' in artist:
        artist = artist.split(' with ')[0]
    elif ' With ' in artist:
        artist = artist.split(' With ')[0]
    elif "," in artist:
        artist = artist.split(',')[0]
    # Make API call
    try:
        # Store search results for track with artist and song title as query parameters
        searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track")
        # Store track ID
        track_id = searchResults['tracks']['items'][0]['id']
        print(f"{song_title} by {artist}")
        return track_id
    
    # Executes if no match found yet
    except:
        # Check for '&' character
        if ' & ' in artist:
            try:
                # Slice artist name before '&' symbol to get first artist name
                artist = artist.split(' & ')[0]
                # Store search results for track with artist and song title as query parameters
                searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track")
                # Store track ID
                track_id = searchResults['tracks']['items'][0]['id']
                print(f"{song_title} by {artist}")
                return track_id
            
            # Executes if still no match found
            except:
                # Print song title and artist for non-match
                print(f'No ID found for {song_title} by {artist}')

In [9]:
def get_audio_features(uri):
    searchResults = sp.audio_features(uri)
    return searchResults

In [78]:
song = "Did You Have A Happy Birthday"
artist = "Paul Anka"
# result = sp.search(q=f"artist: {artist} track: {song}", type="track")['tracks']['items'][0]#['id']
result = sp.search(q=f"{artist} {song}", type=["track", "artist"],)['tracks']['items']#[0]['id']
result
# sp.search(q=f"track:{song} artist:{artist}", type='track', limit=1)
# get_track_id(song, artist)
# len(result)



[{'album': {'album_group': 'album',
   'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7ceUfdWq2t5nbatS6ollHh'},
     'href': 'https://api.spotify.com/v1/artists/7ceUfdWq2t5nbatS6ollHh',
     'id': '7ceUfdWq2t5nbatS6ollHh',
     'name': 'Paul Anka',
     'type': 'artist',
     'uri': 'spotify:artist:7ceUfdWq2t5nbatS6ollHh'}],
   'available_markets': ['CA', 'US'],
   'external_urls': {'spotify': 'https://open.spotify.com/album/3rhYtnqug5utE5hEMTRuZl'},
   'href': 'https://api.spotify.com/v1/albums/3rhYtnqug5utE5hEMTRuZl',
   'id': '3rhYtnqug5utE5hEMTRuZl',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273f4ce5482cb7d519249712290',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e02f4ce5482cb7d519249712290',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d00004851f4ce5482cb7d519249712290',
     'width': 64}],
   'is_playable'

In [12]:
unique_billboard_tracks_df[unique_billboard_tracks_df['song']==song]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1


In [ ]:
# # Import dependency for swifter module


# billboard_1960s['track_id'] = billboard_1960s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
# billboard_1960s.to_csv(f'./Resources/billboard_1960s.csv')

In [79]:
# billboard_1970s['track_id'] = billboard_1970s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
# billboard_1970s.to_csv(f'./Resources/billboard_1970s.csv')


billboard_1980s['track_id'] = billboard_1980s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1980s.to_csv(f'./Resources/billboard_1980s.csv')


billboard_1990s['track_id'] = billboard_1990s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1990s.to_csv(f'./Resources/billboard_1990s.csv')


billboard_2000s['track_id'] = billboard_2000s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2000s.to_csv(f'./Resources/billboard_2000s.csv')


billboard_2010s['track_id'] = billboard_2010s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2010s.to_csv(f'./Resources/billboard_2010s.csv')


billboard_2020s['track_id'] = billboard_2020s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2020s.to_csv(f'./Resources/billboard_2020s.csv')

In [11]:
df_1960s = pd.read_csv('./Resources/billboard_1960s.csv')

In [12]:
df_1960s.head()

,Unnamed: 0,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1,track_id
0,10,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,NaN,2aeZdQUwBksJPaRk1gaEAL
1,11,(A Ship Will Come) Ein Schiff Wird Kommen,Lale Anderson,1961,88,4,NaN,NaN
2,13,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,NaN,6x5BiQwNlbtisITsEHa8Eu
3,19,(Baby) Hully Gully,The Olympics,1960,72,7,NaN,2CkbNJooPxUsHG4gFXKhxa
4,29,(Come round Here) Im The One You Need,The Miracles,1966,17,9,NaN,5YvokzRkNONpj5c0TIqTjQ


In [56]:
# df_1960s[df_1960s['track_id']]


null_rows = df_1960s[df_1960s['track_id'].isnull()]

null_rows[null_rows['song'].str.contains("\?")]
# null_rows

,Unnamed: 0,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1,track_id
500,2042,Are You Lonesome To-night?,Elvis Presley With The Jordanaires,1960,1,7,NaN,NaN
501,2043,Are You Lonesome To-night?,Elvis Presley With The Jordanaires,1961,1,16,NaN,NaN
1408,6599,Did You Have A Happy Birthday?,Paul Anka,1963,89,3,NaN,NaN
1469,6941,Does He Mean That Much To You?,Eddy Arnold,1962,98,1,NaN,NaN
7236,33608,What Would You Do?,Jim Reeves,1961,73,5,NaN,NaN
7333,34116,While Youre Out Looking For Sugar?,The Honey Cone,1969,62,8,NaN,NaN
7363,34269,Who Stole The Keeshka?,The Matys Bros.,1963,55,9,NaN,NaN
7374,34320,Whos Been Sleeping In My Bed?,Linda Scott,1964,100,1,NaN,NaN
7388,34378,Why Did You Leave Me?,Vince Edwards,1962,68,6,NaN,NaN


In [ ]:
df_1960s.drop_duplicates(['artist', 'song'])

,Unnamed: 0,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1,track_id
0,10,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,NaN,2aeZdQUwBksJPaRk1gaEAL
1,11,(A Ship Will Come) Ein Schiff Wird Kommen,Lale Anderson,1961,88,4,NaN,NaN
2,13,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,NaN,6x5BiQwNlbtisITsEHa8Eu
3,19,(Baby) Hully Gully,The Olympics,1960,72,7,NaN,2CkbNJooPxUsHG4gFXKhxa
4,29,(Come round Here) Im The One You Need,The Miracles,1966,17,9,NaN,5YvokzRkNONpj5c0TIqTjQ
...,...,...,...,...,...,...,...,...
7785,36028,Zero-Zero,Lawrence Welk,1962,98,1,NaN,5SUpO1bYMqXn2sjZiFXFI9
7786,36030,Zing! Went The Strings Of My Heart,The Furys,1963,92,2,NaN,NaN
7787,36031,Zip Code,The Five Americans,1967,36,7,NaN,1bdtY3SGI196r2LF0Dd6Oa
7788,36032,Zip-A-Dee Doo-Dah,Bob B. Soxx And The Blue Jeans,1962,9,7,NaN,477feyuGo8JUehyr1GXlVo
